The <a href="https://data.cervantesvirtual.com">Biblioteca Virtual Miguel de Cervantes</a> published its catalogue as Linked Open data using as main vocabulary <a href="http://www.rdaregistry.info/">Resource, Description and Access</a>.

This example shows how to use <a href="https://www.w3.org/TR/sparql11-query/">SPARQL</a> as a query language in Linked Open Data repositories.

### First of all, we init the SPARQLWrapper service with the SPARQL endpoint

In [61]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

### Let's write our first SPARQL sentence

We could be interested in the classes used in the repository

In [62]:
sparql.setQuery("""
PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
PREFIX rdam: <http://rdaregistry.info/Elements/m/>
PREFIX rdae: <http://rdaregistry.info/Elements/e/>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>

SELECT ?author ?work ?workLabel ?placeOfProduction ?yearOfPublication ?langCode 
WHERE {
  wd:Q2085725 wdt:P2799 ?id .
  wd:Q2085725 rdfs:label ?author.  FILTER(LANG(?author) = "en").
  BIND(uri(concat("https://data.cervantesvirtual.com/person/", ?id)) as ?bvmcID)
  SERVICE <http://data.cervantesvirtual.com/openrdf-sesame/repositories/data> {
    ?work rdaw:author ?bvmcID .
    ?work rdfs:label ?workLabel .
    ?work rdaw:manifestationOfWork ?manifestation .  
    ?work rdaw:expressionOfWork ?expression .
    OPTIONAL {?expression rdae:languageOfExpression ?language . ?language madsrdf:code ?langCode .}
    OPTIONAL {?manifestation rdam:placeOfProduction ?placeOfProduction .}
    OPTIONAL {?manifestation rdam:dateOfPublication ?dateOfPublication . BIND(REPLACE(str(?dateOfPublication), "https://data.cervantesvirtual.com/date/", "", "i") AS ?yearOfPublication) .}
  }
}
LIMIT 1000
"""
)

### Now we retrieve the results

In [63]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Author: ', r['author']['value'], ' Work: ', r['work']['value'], ' Title: ', r['workLabel']['value'], ' placeOfProduction: ', r['placeOfProduction']['value'],
             ' Year publication: ', r['yearOfPublication']['value'])
except Exception as e:
    print(e)

Author:  Jorge Juan y Santacilia  Work:  https://data.cervantesvirtual.com/work/662222  Title:  Examen maritimo theorico practico, o tratado de mechanica aplicado a la construccion, conocimiento y manejo de los navios y demas embarcaciones  placeOfProduction:  en la Imprenta de D. Francisco Manuel de Mena, Calle de las Carretas, 1771  Year publication:  1771
Author:  Jorge Juan y Santacilia  Work:  https://data.cervantesvirtual.com/work/666509  Title:  Elogio de D. Jorge Juan, Comendador de Aliaga en la Orden de S. Juan... [Texto impreso]  placeOfProduction:  1750  Year publication:  1750
Author:  Jorge Juan y Santacilia  Work:  https://data.cervantesvirtual.com/work/666530  Title:  Estado de la astronomia en Europa [Texto impreso] : Y juicio de los fundamentos sobre que se erigieron los Systemas del Mundo, para que sirva de guia al mètodo en que debe recibirlos la Nación ...  placeOfProduction:  en la Imprenta Real de la Gazeta, 1774  Year publication:  1774
Author:  Jorge Juan y Sant

### Let's create a dataframe

In [64]:
data = [{"author" : x["author"]["value"], "work" : x["work"]["value"],
        "title" : x["workLabel"]["value"], "placeOfProduction" : x["placeOfProduction"]["value"],
        "yearOfPublication" : int(x["yearOfPublication"]["value"]), 
        "langCode" : x["langCode"]["value"]} for x in ret["results"]["bindings"]]

In [65]:
import pandas as pd

df = pd.DataFrame(data)

In [66]:
df.head(10)

,author,work,title,placeOfProduction,yearOfPublication,langCode
0,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/662222,"Examen maritimo theorico practico, o tratado d...","en la Imprenta de D. Francisco Manuel de Mena,...",1771,es
1,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/666509,"Elogio de D. Jorge Juan, Comendador de Aliaga ...",1750,1750,es
2,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/666530,Estado de la astronomia en Europa [Texto impre...,"en la Imprenta Real de la Gazeta, 1774",1774,es
3,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/666560,Observaciones astronomicas y physicas ... en l...,"por Juan de Zuñiga, 1748",1748,es
4,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/109096,Método de levantar y dirigir el mapa o plano g...,"Madrid, en la Imprenta Real, 1809, pp. 143-151",1809,es
5,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/109097,Reflexiones sobre el Método de levantar el map...,"Madrid, en la Imprenta Real, 1809, pp. 152-155",1809,es
6,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/109098,Carta de don Jorge Juan a don Sebastián Canter...,"Madrid, en la Imprenta Real, 1809, pp. 160-163",1809,es
7,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/109099,Parecer de don Jorge Juan sobre el relox o cro...,"Madrid, en la Imprenta Real, 1809, pp. 176-184",1809,es
8,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/109100,Reflexiones sobre la fábrica y uso del Cuarto-...,"Madrid, en la Imprenta Real, 1809, pp. 253-320",1809,es
9,Jorge Juan y Santacilia,https://data.cervantesvirtual.com/work/97199,Compendio de navegación para el uso de los cav...,España. Cádiz,1757,es


In [67]:
df.describe()

,yearOfPublication
count,38.000000
mean,1773.184211
std,33.344926
min,1748.000000
25%,1749.250000
50%,1764.000000
75%,1788.250000
max,1918.000000


In [68]:
df.dtypes

author               object
work                 object
title                object
placeOfProduction    object
yearOfPublication     int64
langCode             object
dtype: object

In [77]:
sorted_df = df.sort_values(by='yearOfPublication')
print(sorted_df[["title","yearOfPublication"]].head(40))

                                                title  yearOfPublication
3   Observaciones astronomicas y physicas ... en l...               1748
13  Relacion historica del viage a la America Meri...               1748
14  Relacion historica del viage a la America Meri...               1748
15  Relacion historica del viage a la America Meri...               1748
12  Relacion historica del viage a la America Meri...               1748
19  Relacion historica del viage a la America Meri...               1748
27  Observaciones astronomicas y phisicas hechas d...               1748
18  Dissertacion historica, y geographica sobre el...               1749
16  Dissertacion historica, y geographica sobre el...               1749
26  Dissertacion historica, y geographica sobre el...               1749
1   Elogio de D. Jorge Juan, Comendador de Aliaga ...               1750
31  Voyage historique de l'amerique meridionale : ...               1752
20  Voyage historique de l'Amerique Meridionale : .